In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import pandas as pd
import numpy as np
import os
import tensorflow as tf

In [ ]:
path_to_json_converter = "/content/drive/MyDrive/TwitterNLP/ref_converter.json"

with open(path_to_json_converter) as ref_json:
    ref_dict= json.load(ref_json)

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/TwitterNLP/TweetsAndReplies.csv")[:10000]
data = data.dropna()
print(len(data))


9946


In [ ]:
data.head()

,Tweet,Reply
0,Lenin died #OnThisDay 1924. He dismissed our n...,It's true. And the coverage of anything China...
1,There is a “fundamental tension within the cli...,What has The Economist done to protect Pacifi...
2,It is difficult to quantify exactly how much o...,Wonder where the data for that map is from?
3,Are Meta’s problems a one-off or a sign of dee...,Burn baby burn..Facebook is on its way down.A...
4,It is difficult to quantify exactly how much o...,Hi TheEconomists! We have the longest life ex...


In [ ]:
# https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python

import re

def remove_emojis(text):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', text)

In [ ]:
remove_emojis("Hi 🤔 How is your 🙈 and 😌. Have a nice weekend 💕👭👙")

'Hi  How is your  and . Have a nice weekend '

In [ ]:
data.Tweet = data.Tweet.apply(remove_emojis)
data.Reply = data.Reply.apply(remove_emojis)

In [ ]:
def convert_to_numb(tweet_key_pair):
    global ref_dict

    #Function to update the dictionary
    def update_ref_dict(tweet_string):
        with open(path_to_json_converter,'r+') as file:
            dict_data = json.load(file)
            value = max(dict_data.values())

            for item in tweet_string:
                if item not in dict_data:
                    value += 1
                    dict_data[item] = value

            file.seek(0)# Insert at the beginning of the file
            ref_dict = dict_data
            json.dump(ref_dict, file,indent=4)

    # Remove leading and trailing gaps and return all character/letters
    tweet = list(tweet_key_pair[0].strip())
    reply = list(tweet_key_pair[1].strip())

    tweet_result = list(map(ref_dict.get,tweet))
    reply_result = list(map(ref_dict.get,reply))
    
    if None in tweet_result or None in reply_result:
      update_ref_dict(tweet)
      update_ref_dict(reply)
      tweet_result = list(map(ref_dict.get,tweet))
      reply_result = list(map(ref_dict.get,reply))
    
    tweet_result = tweet_result + [0]*(280 - len(tweet_result))
    reply_result = reply_result + [0]*(280 - len(reply_result))

     
    if len(tweet_result)==280 and len(reply_result)==280: 
      tweet_key_pair["Tweet"] = tweet_result
      tweet_key_pair["Reply"] = reply_result
      return tweet_key_pair

In [ ]:
# Use copy to prevent altering the original df
converted_tweets = data.copy().apply(convert_to_numb, axis=1).dropna().reset_index(drop=True)
converted_tweets.head()

,Tweet,Reply
0,"[37, 4, 13, 8, 13, 95, 3, 8, 4, 3, 95, 54, 40,...","[34, 19, 58, 18, 95, 19, 17, 20, 4, 65, 95, 26..."
1,"[34, 19, 95, 8, 18, 95, 3, 8, 5, 5, 8, 2, 20, ...","[48, 14, 13, 3, 4, 17, 95, 22, 7, 4, 17, 4, 95..."
2,"[26, 17, 4, 95, 38, 4, 19, 0, 111, 18, 95, 15,...","[27, 20, 17, 13, 95, 1, 0, 1, 24, 95, 1, 20, 1..."
3,"[34, 19, 95, 8, 18, 95, 3, 8, 5, 5, 8, 2, 20, ...","[33, 8, 95, 45, 7, 4, 30, 2, 14, 13, 14, 12, 8..."
4,"[26, 17, 4, 95, 38, 4, 19, 0, 111, 18, 95, 15,...","[50, 4, 18, 65, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
#converted_tweets.to_csv("/content/drive/MyDrive/TwitterNLP/ConvertedTweets.csv")
#converted_tweets = pd.read_csv("/content/drive/MyDrive/TwitterNLP/ConvertedTweets.csv", index_col=[0])
#converted_tweets.head()

In [ ]:
# To use tf.data.Dataset.from_tensor_slices, you have to pass a multidimensional array
tweet_col = np.array([i for i in converted_tweets["Tweet"]]).astype(float)
reply_col = np.array([i for i in converted_tweets["Reply"]]).astype(float)

In [ ]:
tweet_col.shape

(9917, 280)

In [ ]:
tweet_col

array([[37.,  4., 13., ...,  0.,  0.,  0.],
       [34., 19., 95., ...,  0.,  0.,  0.],
       [26., 17.,  4., ...,  0.,  0.,  0.],
       ...,
       [41., 14., 18., ...,  0.,  0.,  0.],
       [36.,  4., 13., ...,  0.,  0.,  0.],
       [41., 14., 18., ...,  0.,  0.,  0.]])

In [ ]:
#TF dataset
dataset = tf.data.Dataset.from_tensor_slices((tweet_col, reply_col))
dataset

<TensorSliceDataset element_spec=(TensorSpec(shape=(280,), dtype=tf.float64, name=None), TensorSpec(shape=(280,), dtype=tf.float64, name=None))>

In [ ]:
VOCAB_SIZE = len(ref_dict)
BATCH_SIZE = 128
BUFFER_SIZE = 10000
# Number of characters in the output/tweet
TWEET_SIZE = 280 
EMBEDDING_DIM = 512
RNN_UNITS = 200 #1024
EPOCHS = 10

In [ ]:
dataset_to_use = dataset.batch(batch_size = BATCH_SIZE, drop_remainder=True)

In [ ]:
dataset_to_use.element_spec

(TensorSpec(shape=(128, 280), dtype=tf.float64, name=None),
 TensorSpec(shape=(128, 280), dtype=tf.float64, name=None))

In [ ]:
def build_model(vocab_size, rnn_units, embedding_dim, batch_size):
  model=tf.keras.Sequential([
                             tf.keras.layers.Embedding(input_dim = vocab_size, 
                                                       output_dim = embedding_dim, 
                                                       batch_input_shape=[batch_size,None]),
                             tf.keras.layers.LSTM(units = rnn_units,
                                                 stateful = True,
                                                 return_sequences=True,
                                                 dropout = 0.2),
                             tf.keras.layers.LSTM(units = rnn_units,
                                                 stateful = True,
                                                 return_sequences=True,
                                                 dropout = 0.2),
                             tf.keras.layers.Dense(VOCAB_SIZE)
  ])
  return model

In [ ]:
model = build_model(
    vocab_size = VOCAB_SIZE,
    rnn_units = RNN_UNITS, 
    embedding_dim = EMBEDDING_DIM,
    batch_size = BATCH_SIZE
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (128, None, 512)          244224    
                                                                 
 lstm_4 (LSTM)               (128, None, 200)          570400    
                                                                 
 lstm_5 (LSTM)               (128, None, 200)          320800    
                                                                 
 dense_2 (Dense)             (128, None, 477)          95877     
                                                                 
Total params: 1,231,301
Trainable params: 1,231,301
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.output

<KerasTensor: shape=(128, None, 477) dtype=float32 (created by layer 'dense_2')>

In [ ]:
# Train the model and create checkpoints
def model_loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=model_loss)

#creat checpoint
checkpoint_dir ="./training_checkpoint"

# name the checpoint files
checkpoint_prefix = os.path.join(checkpoint_dir,'ckpt_{epoch}') 

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                      filepath = checkpoint_prefix,
                      save_weights_only = True,)

history = model.fit(dataset_to_use, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
77/77 [==============================] - 24s 261ms/step - loss: nan
Epoch 2/10
20/77 [======>.......................] - ETA: 14s - loss: nan

KeyboardInterrupt: ignored

In [ ]:
model = build_model(
    vocab_size = VOCAB_SIZE,
    rnn_units = RNN_UNITS, 
    embedding_dim = EMBEDDING_DIM,
    batch_size = 1
)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [ ]:
''' Input tweet can be None incase we want it to come 
up with a tweet or give an input tweet if we want a reply
For now we only assume it takes in tweets and gives replies
'''
def generate_reply(model,input_tweet, tweet_size =TWEET_SIZE):
  global ref_dict

  # Number of characters to generate
  input_values = list(map(ref_dict.get,input_tweet))
  input_values = tf.expand_dims(input_values,axis=0)

  # Empty list for generated result
  idx_reply = []
  generated_reply = []
  ''' 
  High temp = More surprising text
  Low temperature = More predictable text
  '''
  temperature =0.8

  # Here batch size is 1
  model.reset_states()

  for _ in range(tweet_size):
    prediction = model(input_values)
    prediction = tf.squeeze(prediction,0)
    prediction = prediction/temperature

    predicted_id = tf.random.categorical(prediction, num_samples=1)[-1,0].numpy()
    input_values = tf.expand_dims([predicted_id],axis=0)

    idx_reply.append(predicted_id)

  
  #idx_reply = predicted_ids.numpy().ravel()

  for i in idx_reply:
    generated_character = list(ref_dict.keys())[list(ref_dict.values()).index(i)]
    if generated_character == "null":
      generated_character=""
      
    generated_reply.append(generated_character)
    

  return "".join(generated_reply)


In [ ]:
reply = generate_reply(model,"I think the issue may be more common than you think, ama namna gani ?")
reply

'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'

In [ ]:
len(reply)

280

In [ ]:
MODEL_DIR = "/content/drive/MyDrive/TwitterNLP"
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite = True,
    include_optimizer = True,
    save_format = None,
    signatures = None,
    options = None
)
print("\nSaved model:")
!ls -l {export_path}


export path = /content/drive/MyDrive/TwitterNLP/1



INFO:tensorflow:Assets written to: /content/drive/MyDrive/TwitterNLP/1/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/TwitterNLP/1/assets



Saved model:
total 1436
drwx------ 2 root root    4096 Mar  4 17:22 assets
-rw------- 1 root root   13822 Mar 20 12:06 keras_metadata.pb
-rw------- 1 root root 1448238 Mar 20 12:06 saved_model.pb
drwx------ 2 root root    4096 Mar 20 12:06 variables


In [ ]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['embedding_1_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (1, -1)
        name: serving_default_embedding_1_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (1, -1, 2223)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Concrete Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          

In [ ]:
new_model = tf.keras.models.load_model(export_path)

In [ ]:
# Check its architecture
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 512)            1138176   
                                                                 
 lstm_2 (LSTM)               (1, None, 200)            570400    
                                                                 
 lstm_3 (LSTM)               (1, None, 200)            320800    
                                                                 
 dense_1 (Dense)             (1, None, 2223)           446823    
                                                                 
Total params: 2,476,199
Trainable params: 2,476,199
Non-trainable params: 0
_________________________________________________________________


In [ ]:
reply = generate_reply(new_model, "Ona sasa")
print(len(reply),"\n",reply)

7 
  o ’tfa
